In [31]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [60]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        main_dataset = []
        cats = []
        for i in range(len(temp_dataset)):
            # store cats
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                cats.append([img, torch.tensor([0])])

            # add other images to dataset
            else:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                main_dataset.append([img, torch.tensor([1])])

        # Repeat dataset2 nine times
        repeated_datasets = [cats] * 9

        # Concatenate dataset1 with repeated dataset2
        self.dataset = ConcatDataset([main_dataset] + repeated_datasets)

    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        image = torchvision.transforms.RandomResizedCrop((32, 32), scale=(4/5, 5/4), ratio=(4/5, 5/4))(image)
        return (image, label)

In [61]:
train_dataset = CatsDataset(train=True)
len(train_dataset)

90000

In [62]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True
)

In [63]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [77]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(64, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(64, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(256, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [84]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

In [86]:
for epoch in range(32):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.39095981749282643
Epoch 2 Loss: 0.38968457925048744
Epoch 3 Loss: 0.38923291726545856


KeyboardInterrupt: 

In [67]:
test_dataset = CatsDataset(train=False)

In [68]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=True
)

In [87]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.8039, device='mps:0')
